In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import time
from shapely.geometry import Point
from geopandas.tools import sjoin
from skimpy import skim

In [3]:
# Import of the Polygons
gdf_combined = gpd.read_file("C:/Users/edoar/combined_quartieri.geojson")

# YELP API

In [6]:
neighborhoods = [f"{name}, Milan" for name in gdf_combined["Neighborhood"].tolist()]
neighborhoods

['Parco delle Abbazie, Milan',
 'Adriano, Milan',
 'Affori, Milan',
 'Baggio, Milan',
 'Bande Nere, Milan',
 'Barona, Milan',
 'Bicocca, Milan',
 'Bovisasca, Milan',
 'Bovisa, Milan',
 'Brera, Milan',
 'Bruzzano, Milan',
 'Buenos Aires - Venezia, Milan',
 'Cantalupa, Milan',
 'Cascina Triulza - Expo, Milan',
 'Centrale, Milan',
 'Chiaravalle, Milan',
 'Città Studi, Milan',
 'Comasina, Milan',
 'Corsica, Milan',
 'De Angeli - Monte Rosa, Milan',
 'Dergano, Milan',
 'Duomo, Milan',
 'Ex Om - Morivione, Milan',
 'Farini, Milan',
 'GARIBALDI REPUBBLICA, Milan',
 'Gallaratese, Milan',
 'Ghisolfa, Milan',
 'Giambellino, Milan',
 'Giardini Porta Venezia, Milan',
 'Gratosoglio - Ticinello, Milan',
 'Greco, Milan',
 'Guastalla, Milan',
 'Isola, Milan',
 'Lambrate, Milan',
 'Lodi - Corvetto, Milan',
 'Lorenteggio, Milan',
 'Loreto, Milan',
 'Maciachini - Maggiolina, Milan',
 'Magenta - San Vittore, Milan',
 'Maggiore - Musocco, Milan',
 'Mecenate, Milan',
 'Muggiano, Milan',
 'Navigli, Milan',
 

In [8]:
# Replace with your Yelp API key
API_KEY = "6CbFJGYPopltfBbkb00hIFGRI4XBrAsccPevTJ53ol4YIuJrF48kEylTmKvEl4-TEP8p0pEc3ydzUsatpgGI5aqSHCsysi5-yWyeJD3V-6al5x6_AcFkw23LehKEZ3Yx"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

# Constants
businesses_per_request = 48  # Maximum allowed by Yelp per request

def make_request(url, params=None):
    """
    Makes a request to the Yelp API.
    """
    response = requests.get(url, headers=HEADERS, params=params)
    response.raise_for_status()
    return response

def search_businesses(location, term="nightlife", limit=businesses_per_request, offset=0):
    """
    Searches for nightlife businesses in a given location with pagination.
    """
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "location": location,
        "term": term,
        "limit": limit,
        "offset": offset
    }
    response = make_request(url, params=params)
    return response.json().get("businesses", [])

# Initialize data storage
data = []

try:
    for neighborhood in neighborhoods:  # Loop through each neighborhood
        offset = 0
        while True:  # Keep fetching data until no more results
            print(f"Fetching nightlife in {neighborhood} with offset: {offset}...")

            try:
                # fetch businesses using the current offset and location
                businesses = search_businesses(location=neighborhood, term="nightlife", limit=businesses_per_request, offset=offset)

                if not businesses:
                    # No more businesses to fetch
                    print(f"No more nightlife venues returned for {neighborhood}.")
                    break

                for biz in businesses:
                    name = biz.get("name", None)
                    location_info = biz.get("location", {})
                    address = location_info.get("address1", None)
                    categories = biz.get("categories", [])
                    category_list = [cat.get("title", "") for cat in categories if cat.get("title")]
                    category_str = ", ".join(category_list) if category_list else None
                    rating = biz.get("rating", None)
                    review_count = biz.get("review_count", None)

                    # Extract coordinates
                    coordinates = biz.get("coordinates", {})
                    latitude = coordinates.get("latitude", None)
                    longitude = coordinates.get("longitude", None)

                    # Append to data
                    data.append({
                        "Fetch Location": neighborhood,
                        "Venue Name": name,
                        "Venue Address": address,
                        "Categories": category_str,
                        "Average Star Rating": rating,
                        "Review Count": review_count,
                        "Latitude": latitude,
                        "Longitude": longitude
                    })

                # Increment offset for the next batch
                offset += len(businesses)

                # Optional: Sleep to respect API rate limits
                time.sleep(0.5)

                # Break if the offset exceeds Yelp's maximum results per query
                if offset >= 240:  # Maximum 240 results per query
                    print(f"Reached maximum results for {neighborhood}.")
                    break

            except requests.HTTPError as he:
                # Log the error and skip this neighborhood
                print(f"HTTP error occurred for {neighborhood}: {he}")
                break

    # Convert the collected data into a DataFrame
    df = pd.DataFrame(data)

    # Convert DataFrame to GeoDataFrame
    df["geometry"] = df.apply(
        lambda row: Point(row["Longitude"], row["Latitude"]) if row["Longitude"] and row["Latitude"] else None,
        axis=1
    )
    Nightlife = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

    # Print summary
    print(Nightlife.head())
    print(f"Total nightlife venues collected: {len(data)}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

Fetching nightlife in Parco delle Abbazie, Milan with offset: 0...
Fetching nightlife in Parco delle Abbazie, Milan with offset: 48...
Fetching nightlife in Parco delle Abbazie, Milan with offset: 96...
Fetching nightlife in Parco delle Abbazie, Milan with offset: 144...
Fetching nightlife in Parco delle Abbazie, Milan with offset: 192...
Reached maximum results for Parco delle Abbazie, Milan.
Fetching nightlife in Adriano, Milan with offset: 0...
HTTP error occurred for Adriano, Milan: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=Adriano%2C+Milan&term=nightlife&limit=48&offset=0
Fetching nightlife in Affori, Milan with offset: 0...
Fetching nightlife in Affori, Milan with offset: 48...
Fetching nightlife in Affori, Milan with offset: 96...
Fetching nightlife in Affori, Milan with offset: 144...
Fetching nightlife in Affori, Milan with offset: 192...
Reached maximum results for Affori, Milan.
Fetching nightlife in Baggio, Milan with offset: 

In [10]:
Nightlife

,Fetch Location,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,Latitude,Longitude,geometry
0,"Parco delle Abbazie, Milan",Twiggy Club,Via de Cristoforis 5,"Music Venues, Pubs, Cafes",5.0,1,45.821506,8.831431,POINT (8.83143 45.82151)
1,"Parco delle Abbazie, Milan",Crazy Pub,Via Torino 20,"Italian, Burgers, Pubs",4.5,8,45.674136,8.741011,POINT (8.74101 45.67414)
2,"Parco delle Abbazie, Milan",H2O Live Club,Via Mezzomerico 18,Dance Clubs,0.0,0,45.626890,8.625470,POINT (8.62547 45.62689)
3,"Parco delle Abbazie, Milan",Movie,Via Tripoli 21,Dance Clubs,5.0,1,45.616225,9.205655,POINT (9.20566 45.61622)
4,"Parco delle Abbazie, Milan",049,Piazza Martiri della Libertà 3B,"Lounges, Wine Bars, Cocktail Bars",4.7,3,45.445177,8.616799,POINT (8.6168 45.44518)
...,...,...,...,...,...,...,...,...,...
17827,"Forze Armate, Milan",BAR Metro di D' Ambrosio Giorgio,"Via Martinitt, 3",Bars,0.0,0,45.465980,9.149540,POINT (9.14954 45.46598)
17828,"Forze Armate, Milan",Scooby Doo,Via Giuseppe Parini 15,Bars,3.0,1,45.430024,9.112877,POINT (9.11288 45.43002)
17829,"Forze Armate, Milan",Caffè Rossini,Via Mario Pagano 69,"Cocktail Bars, Sandwiches, Breakfast & Brunch",0.0,0,45.468052,9.160705,POINT (9.1607 45.46805)
17830,"Forze Armate, Milan",Al Nazionale...Naturalmente,Piazza Piemonte 12,"Bistros, Breakfast & Brunch, Cocktail Bars",0.0,0,45.466710,9.154190,POINT (9.15419 45.46671)


In [12]:
# Removing the dupes
Nightlife_nodup = Nightlife.drop_duplicates(subset=['Venue Name','Venue Address','Categories','Average Star Rating',
                                                        'Review Count','geometry'])
Nightlife_nodup

,Fetch Location,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,Latitude,Longitude,geometry
0,"Parco delle Abbazie, Milan",Twiggy Club,Via de Cristoforis 5,"Music Venues, Pubs, Cafes",5.0,1,45.821506,8.831431,POINT (8.83143 45.82151)
1,"Parco delle Abbazie, Milan",Crazy Pub,Via Torino 20,"Italian, Burgers, Pubs",4.5,8,45.674136,8.741011,POINT (8.74101 45.67414)
2,"Parco delle Abbazie, Milan",H2O Live Club,Via Mezzomerico 18,Dance Clubs,0.0,0,45.626890,8.625470,POINT (8.62547 45.62689)
3,"Parco delle Abbazie, Milan",Movie,Via Tripoli 21,Dance Clubs,5.0,1,45.616225,9.205655,POINT (9.20566 45.61622)
4,"Parco delle Abbazie, Milan",049,Piazza Martiri della Libertà 3B,"Lounges, Wine Bars, Cocktail Bars",4.7,3,45.445177,8.616799,POINT (8.6168 45.44518)
...,...,...,...,...,...,...,...,...,...
17763,"Forze Armate, Milan",Mazzonna / Antonio,"Piazza Giovanni Xxiii, 7",Bars,0.0,0,45.439020,9.109670,POINT (9.10967 45.43902)
17764,"Forze Armate, Milan",BAR Piccolo Lord di Comisso,"Via Della Liberazione, 34/A",Bars,0.0,0,45.439727,9.104614,POINT (9.10461 45.43973)
17770,"Forze Armate, Milan",Bonavetti / Andrea,"Via Sebastiano Caboto, 31",Bars,0.0,0,45.443188,9.114465,POINT (9.11447 45.44319)
17774,"Forze Armate, Milan",Cafe' Principe di Massa Antonella,"Via Roma, 108",Bars,0.0,0,45.439091,9.099020,POINT (9.09902 45.43909)


In [14]:
# Have to be the same type
print(Nightlife_nodup.crs)  # Restaurants CRS
print(gdf_combined.crs)  # Neighborhood polygons CRS

EPSG:4326
EPSG:4326


# Spatial Join

In [17]:
# Perform the spatial join
joined = sjoin(Nightlife_nodup, gdf_combined, how="left", predicate="within")

# Check the result
joined

,Fetch Location,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,Latitude,Longitude,geometry,index_right,Neighborhood
0,"Parco delle Abbazie, Milan",Twiggy Club,Via de Cristoforis 5,"Music Venues, Pubs, Cafes",5.0,1,45.821506,8.831431,POINT (8.83143 45.82151),NaN,NaN
1,"Parco delle Abbazie, Milan",Crazy Pub,Via Torino 20,"Italian, Burgers, Pubs",4.5,8,45.674136,8.741011,POINT (8.74101 45.67414),NaN,NaN
2,"Parco delle Abbazie, Milan",H2O Live Club,Via Mezzomerico 18,Dance Clubs,0.0,0,45.626890,8.625470,POINT (8.62547 45.62689),NaN,NaN
3,"Parco delle Abbazie, Milan",Movie,Via Tripoli 21,Dance Clubs,5.0,1,45.616225,9.205655,POINT (9.20566 45.61622),NaN,NaN
4,"Parco delle Abbazie, Milan",049,Piazza Martiri della Libertà 3B,"Lounges, Wine Bars, Cocktail Bars",4.7,3,45.445177,8.616799,POINT (8.6168 45.44518),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17763,"Forze Armate, Milan",Mazzonna / Antonio,"Piazza Giovanni Xxiii, 7",Bars,0.0,0,45.439020,9.109670,POINT (9.10967 45.43902),NaN,NaN
17764,"Forze Armate, Milan",BAR Piccolo Lord di Comisso,"Via Della Liberazione, 34/A",Bars,0.0,0,45.439727,9.104614,POINT (9.10461 45.43973),NaN,NaN
17770,"Forze Armate, Milan",Bonavetti / Andrea,"Via Sebastiano Caboto, 31",Bars,0.0,0,45.443188,9.114465,POINT (9.11447 45.44319),NaN,NaN
17774,"Forze Armate, Milan",Cafe' Principe di Massa Antonella,"Via Roma, 108",Bars,0.0,0,45.439091,9.099020,POINT (9.09902 45.43909),NaN,NaN


In [19]:
joined[joined['Venue Name']=='Alchimia'] # correct

,Fetch Location,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,Latitude,Longitude,geometry,index_right,Neighborhood
5821,"Gratosoglio - Ticinello, Milan",Alchimia,Via Francesco Brioschi 17,"Lounges, Steakhouses",3.9,17,45.446007,9.18065,POINT (9.18065 45.44601),73.0,Ticinese


### As for the restaurants and the museums, also in this case datapoints have been retrieved from locations outside the polygons of the neighborhoods, thus we removed the rows where the neighborhood name is missing

In [22]:
# Remove rows where the geometry is NaN
PolyNightlife = joined[~joined["Neighborhood"].isna()]

# Reset the index if needed
PolyNightlife = PolyNightlife.reset_index(drop=True)

# Keeping only relevant variables
PolyNightlife = PolyNightlife[["Venue Name", "Venue Address", "Categories", "Average Star Rating",
                           "Review Count", "geometry", "Neighborhood"]]
# Print the cleaned GeoDataFrame
PolyNightlife

,Venue Name,Venue Address,Categories,Average Star Rating,Review Count,geometry,Neighborhood
0,Monkey,Via Napo Torriani 5,Cocktail Bars,4.8,30,POINT (9.20352 45.48085),Centrale
1,Circle,Via Stendhal 34,"Lounges, Dance Clubs",3.4,7,POINT (9.15944 45.45343),Tortona
2,Maremilano,Via Fratelli Zoia 10,Nightlife,5.0,1,POINT (9.10812 45.47635),Quarto Cagnino
3,Blue Eyes,Via Luigi De Andreis 13,Dance Clubs,3.0,2,POINT (9.23504 45.46169),Corsica
4,Just Cavalli,Torre Branca,"Dance Clubs, Italian, Cocktail Bars",2.9,30,POINT (9.17289 45.47325),Parco Sempione
...,...,...,...,...,...,...,...
1830,Bar Giorgio,via bezzecca 2,Bars,4.0,1,POINT (9.20958 45.46092),XXII Marzo
1831,Leuci Roberto,"C. Indipendenza, 5",Bars,0.0,0,POINT (9.21226 45.46831),Buenos Aires - Venezia
1832,Spiller,Via Antonio Fogazzaro 23,"Breweries, Pubs",1.5,4,POINT (9.20943 45.45759),XXII Marzo
1833,Bar Roby,Corso Indipendenza 5,Bars,0.0,0,POINT (9.21311 45.4683),Buenos Aires - Venezia


### In this case, as was for the museum data, only two entries have a missing value (Category), thus ensuring the data retrieved can be considered of high quality

In [24]:
skim(PolyNightlife)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1835   │ │ string      │ 4     │                                                          │
│ │ Number of columns │ 6      │ │ float64     │ 1     │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA  ┃ NA %   ┃ mean    ┃ sd      ┃ p0  ┃ p25   ┃ p50  ┃ p75  ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ Average Star Rating       │   0 │      0 │   2.297 │   1.921 │   0 │     0 │    3 │    4 │     5 │ ▇ ▁▃▅▃  │  │
│ │ Review Count              │   0 │      0 │   3.788 │   10.66 │   0 │     0 │    1 │    3 │   316 │    ▇    │  │
│ └───────────────────────────┴─────┴────────┴─────────┴─────────┴─────┴───────┴──────┴──────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                  ┃ NA     ┃ NA %       ┃ words per row               ┃ total words             ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Venue Name                   │      0 │          0 │                         2.3 │                    4274 │  │
│ │ Venue Address                │      0 │          0 │                         3.7 │                    6707 │  │
│ │ Categories                   │      2 │       0.11 │                         2.3 │                    4287 │  │
│ │ Neighborhood                 │      0 │          0 │                         1.7 │                    3098 │  │
│ └──────────────────────────────┴────────┴────────────┴─────────────────────────────┴─────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [27]:
# Saving the file
PolyNightlife.to_file("PolyNightlife.geojson", driver="GeoJSON")